In [1]:
import pandas as pd
import numpy as np
import random as rd
import warnings, glob, os, sys, shutil
from tqdm.notebook import tqdm
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

In [2]:
# 過濾非生理數據與未使用特徵
drop_cols = ['洗腎紀錄時間', '急診門診住院', '生日', '算年齡', 
             'DLC', 'PERMCATH', 'HBs', 'HBsAg', 'HBc', 
             
             '體重2標準體重', '體重3應脫水', '體重結束', '分母體重', '體重機器顯示', 
             'BUN (洗腎後)', 
             'sbp', 'dbp', 'New Intra SBP', 'New Intra DBP',
             'New 處置其他+症狀處置', 'New 處置其他結束', '處置其他結束', '處置其他', '症狀處置', 'NLP Judge', 'BP Judge']

select_cols =  ['性別', '入院方式', 'HCV','HBV','有無糖尿病','FISTULA','GRAFT','Catheter','Intact PTH','age', '體重1開始','開始血壓SBP', '開始血壓DBP',
            '開始脈搏', '體溫', '體重實際脫水','每公斤脫水量(ml/kg)','BUN','K', 'HGB','URR%','Na', 'Ca','P',
            '透析液 Ca','ALBUMIN','ALT (SGPT)','Alk.phosphatase','Ferritin','IRON/TIBC','MCV', 'MCHC', 'MCH','Iron','Glucose AC','RBC', 'WBC',
            'Platelet', 'Creatinine','AST (SGOT)','TIBC','Bilirubin-T', 'Cholesterol-T', 'CRP']
select_cols = select_cols+ ['Max Diff mbp', 'Max Diff sbp','結束脈搏','Final Judge','Raw Index','ID','洗腎紀錄時間去時分']

mask_cols = ['Max Diff mbp','Max Diff sbp', 'Final Judge','結束脈搏'] #'每公斤脫水量(ml/kg)'

In [3]:
# CV process 
ths_days = 30 #change 30 => 90
stack_nums = 12 #max = 12 / 16 lv.3=16, 8, 4
input_nums = 8

final_cv_save_path = './dataset/TimeSeries_TD{}_SN{}_IN{}_CV_[2025-04-25]'.format(ths_days, stack_nums, input_nums)
os.makedirs(final_cv_save_path, exist_ok=True) #創建儲存路徑

def data_ts_stack(group_data_type, group_ID_df, columns, ts_save_path):
    for idx__, (group_name, group_data) in enumerate(tqdm(group_ID_df)):
        # if group_data.ID.unique()[0] in ['07339W939', '07879B585']:
        # if idx__ in [0,1,2,3,4,5]:
        if True:
            timeseries_df = pd.DataFrame([], columns=columns)
            timeseries_df['Group Type'] = np.nan #標記Train、Valid、Test
            group_data = group_data.sort_values(by=['洗腎紀錄時間去時分'], ascending=True)
            group_data = group_data.reset_index(drop=True)
            stop_tip = False
            for idx in group_data.index:
                slice_df = group_data.loc[idx:idx+(stack_nums-1)].copy()
                
                # print(slice_df.shape)
                # if (group_data.shape[0]==stack_nums) and (slice_df.shape[0]==stack_nums) and (abs(slice_df['洗腎紀錄時間去時分'].iloc[0] - slice_df['洗腎紀錄時間去時分'].iloc[-1]).days<=ths_days): #判斷資料是否有{stack_nums}筆，以及區間時間是否小於等於{ths_days}
                if (slice_df.shape[0]==stack_nums) and (abs(slice_df['洗腎紀錄時間去時分'].iloc[0] - slice_df['洗腎紀錄時間去時分'].iloc[-1]).days<=ths_days): #判斷資料是否有{stack_nums}筆，以及區間時間是否小於等於{ths_days}
                    # print(group_data.shape[0])
                    if stack_nums!=input_nums:
                        slice_df = slice_df.iloc[(stack_nums-input_nums)::]
                        slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                    else: 
                        slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                    slice_df.iloc[-2,slice_df.columns.get_indexer(mask_cols)] = -9999
                    timeseries_df = pd.concat([timeseries_df,slice_df])
                # elif group_data.shape[0]<=input_nums:
                else:
                    if stop_tip==False and group_data.shape[0]<stack_nums:
                        stop_tip = True
                        add_rows_num = input_nums - slice_df.shape[0]
                        # print(slice_df['ID'].iloc[0], add_rows_num, stack_nums-input_nums, slice_df.shape)
                        if add_rows_num!=0 and add_rows_num>0:
                            stop_tip = True
                            first_data = slice_df.iloc[0]
                            stacked_rows = pd.concat([slice_df.iloc[0]] * add_rows_num, ignore_index=True, axis=1).T
                            stacked_rows[:] = 9999
                            stacked_rows['ID'] = first_data['ID']
                            stacked_rows['洗腎紀錄時間去時分'] = first_data['洗腎紀錄時間去時分']
                            # print("ADD:", stacked_rows.shape)
                            temp_df = pd.concat([stacked_rows, slice_df], axis=0)
                            temp_df = pd.concat([temp_df,temp_df.iloc[-1].to_frame().T.copy()])
                            temp_df.iloc[-2,temp_df.columns.get_indexer(mask_cols)] = -9999
                            if len(temp_df)!=input_nums+1:
                                print(0," Error Data:",group_data.ID.unique(), temp_df.shape)
                        elif slice_df.shape[0] ==input_nums:
                            temp_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                            temp_df.iloc[-2,temp_df.columns.get_indexer(mask_cols)] = -9999      
                            if len(temp_df)!=input_nums+1:
                                print(1," Error Data:",group_data.ID.unique(), temp_df.shape)
                        elif add_rows_num<0:
                            # temp_df = slice_df.iloc[(stack_nums-input_nums)::]
                            temp_df = slice_df.iloc[(add_rows_num*-1)::]
                            temp_df = pd.concat([temp_df,temp_df.iloc[-1].to_frame().T.copy()])
                            # print(add_rows_num, stack_nums-input_nums, slice_df.shape, temp_df.shape, temp_df['ID'].iloc[0],temp_df['Raw Index'])
                            temp_df.iloc[-2,temp_df.columns.get_indexer(mask_cols)] = -9999
                            if len(temp_df)!=input_nums+1:
                                print(2," Error Data:",group_data.ID.unique(), temp_df.shape)
                        timeseries_df = pd.concat([timeseries_df,temp_df])
                        # print("New ADD shape:", timeseries_df.shape)
                    # else:
                    #     timeseries_df = group_data.head(input_nums)
            if timeseries_df.shape[0]!=0:
                timeseries_df.to_csv(os.path.join(ts_save_path,f'{group_data_type}-{str(group_name[0])}.csv'),encoding='utf-8-sig', index=False)
            else:
                print("Error Data:",group_data.ID.unique())

In [4]:
Data_ = {
    # 'A':'./dataset/DATA_A_FinalFinished[2024-09-27-filter-age]fixed.csv',
    # 'B':'./dataset/DATA_B_FinalFinished[2024-09-27-filter-age]fixed.csv',
    'C':'./dataset/DATA_C_FinalFinished[2024-09-27-filter-age]fixed.csv',
    'ET':'./dataset/DATA_test[2024-09-27_from_DATA_A].csv'
    }
# Data_ = {
    # 'A':'./dataset/DATA_A_FinalFinished[2024-09-27-filter-age]fixed.csv',
    # 'B':'./dataset/DATA_B_FinalFinished[2024-10-10-filger-age].csv',
    # 'C':'./dataset/DATA_C_FinalFinished[2024-10-10-filter-age].csv',
    # 'ET':'./dataset/DATA_test[2024-09-27_from_DATA_A].csv'
    # }

In [5]:
for DATA_NAME in Data_:
    filter_data_df = pd.read_csv(Data_[DATA_NAME])
    filter_data_df = filter_data_df[select_cols]
    filter_data_df['洗腎紀錄時間去時分'] = pd.to_datetime(filter_data_df['洗腎紀錄時間去時分']).dt.date
    
    # for cv_fold in ['fold_0','fold_1', 'fold_2', 'fold_3', 'fold_4']:
    ts_save_path = './dataset/CV_temp_2/'
    if os.path.exists(ts_save_path):
        shutil.rmtree(ts_save_path)
    os.makedirs(ts_save_path, exist_ok=True) #創建儲存路徑
    group_ID_filter_data_df = filter_data_df.groupby(['ID'])
    data_ts_stack('TrainVal', group_ID_filter_data_df, filter_data_df.columns, ts_save_path)
    
    patient_csv_glob = glob.glob(os.path.join(ts_save_path,'*'))
    # trainval_df = pd.DataFrame([], columns=valid_df.columns)
    trainval_df = pd.DataFrame([], columns=filter_data_df.columns)
    for csv_path in tqdm(patient_csv_glob):
        data_df = pd.read_csv(csv_path)
        trainval_df = pd.concat([trainval_df,data_df])
        del data_df
    # save_name = "TrainVal_{}_DATA-{}.csv".format(cv_fold, DATA_NAME)
    save_name = "InternalTest_DATA-{}.csv".format( DATA_NAME)
    trainval_df.to_csv(os.path.join(final_cv_save_path, save_name), encoding='utf-8-sig', index=False)    
    print(trainval_df.shape)
    del trainval_df


  0%|          | 0/508 [00:00<?, ?it/s]

Error Data: ['00E29U959']
Error Data: ['01809M181']
Error Data: ['01945R090']
Error Data: ['01E59Y050']
Error Data: ['02259W121']
Error Data: ['03E84A151']
Error Data: ['050117101']
Error Data: ['05849G181']
Error Data: ['06309J131']
Error Data: ['064897545']
Error Data: ['071795313']
Error Data: ['07403C141']
Error Data: ['07825B585']
Error Data: ['09431M343']
Error Data: ['097251373']
Error Data: ['098969585']
Error Data: ['09E0G1252']
Error Data: ['134811545']


  0%|          | 0/490 [00:00<?, ?it/s]

(1310499, 52)


  0%|          | 0/216 [00:00<?, ?it/s]

Error Data: ['03E36B555']
Error Data: ['076717262']
